In [1]:
pip install qiskit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install qiskit-aer

Note: you may need to restart the kernel to use updated packages.


In [1]:
dataset_path = "/Users/hrudaydivakaran/Desktop/Quantum_proj/UCMerced_LandUse/UCMerced_LandUse/Images_converted"

In [2]:
import os

print(os.listdir(dataset_path))

['forest', 'buildings', 'river', 'mobilehomepark', '.DS_Store', 'harbor', 'golfcourse', 'agricultural', 'runway', 'baseballdiamond', 'overpass', 'chaparral', 'tenniscourt', 'intersection', 'airplane', 'parkinglot', 'sparseresidential', 'mediumresidential', 'denseresidential', 'beach', 'freeway', 'storagetanks']


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report
from qiskit import QuantumCircuit
from qiskit_aer import Aer
import qiskit.quantum_info as qi

dataset_path = "UCMerced_LandUse/UCMerced_LandUse/Images_converted"
classes = sorted(os.listdir(dataset_path))

In [4]:
def preprocess_image(image_path, target_size=(128, 128)):
    img = Image.open(image_path).convert('RGB')
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0
    return img_array

In [5]:
def extract_classical_features(img_array):
    r_hist, _ = np.histogram(img_array[:,:,0].flatten(), bins=32, range=(0,1))
    g_hist, _ = np.histogram(img_array[:,:,1].flatten(), bins=32, range=(0,1))
    b_hist, _ = np.histogram(img_array[:,:,2].flatten(), bins=32, range=(0,1))
    
    r_hist = r_hist / np.sum(r_hist)
    g_hist = g_hist / np.sum(g_hist)
    b_hist = b_hist / np.sum(b_hist)
    
    gray = np.mean(img_array, axis=2)
    dx = np.diff(gray, axis=1)
    dy = np.diff(gray, axis=0)
    
    gradient_magnitude = np.sqrt(dx[:-1,:]**2 + dy[:,:-1]**2)
    edge_hist, _ = np.histogram(gradient_magnitude.flatten(), bins=16, range=(0,0.5))
    edge_hist = edge_hist / np.sum(edge_hist)
    
    channel_means = [np.mean(img_array[:,:,i]) for i in range(3)]
    channel_stds = [np.std(img_array[:,:,i]) for i in range(3)]
    
    color_ratio1 = np.mean(img_array[:,:,0] / (img_array[:,:,1] + 0.01))
    color_ratio2 = np.mean(img_array[:,:,0] / (img_array[:,:,2] + 0.01))
    color_ratio3 = np.mean(img_array[:,:,1] / (img_array[:,:,2] + 0.01))
    
    texture_features = sift_like_features(gray)
    
    return np.concatenate([r_hist, g_hist, b_hist, edge_hist, channel_means, channel_stds, 
                           [color_ratio1, color_ratio2, color_ratio3], texture_features])

In [6]:
def sift_like_features(gray_img, num_features=20):
    h, w = gray_img.shape
    step_y, step_x = h // 4, w // 4
    features = []
    
    for i in range(0, h - step_y, step_y):
        for j in range(0, w - step_x, step_x):
            patch = gray_img[i:i+step_y, j:j+step_x]
            features.append(np.mean(patch))
            features.append(np.std(patch))
    
    return np.array(features)

In [7]:
def shor_inspired_quantum_circuit(data_to_encode, num_qubits=10):
    qc = QuantumCircuit(num_qubits)
    
    for i in range(num_qubits):
        qc.h(i)
    
    feature_idx = 0
    max_features = min(num_qubits * 3, len(data_to_encode))
    
    for layer in range(3):
        for i in range(num_qubits):
            if feature_idx < max_features:
                qc.ry(data_to_encode[feature_idx] * np.pi, i)
                feature_idx = (feature_idx + 1) % max_features
        
        for i in range(num_qubits-1):
            qc.cx(i, i+1)
        
        for i in range(num_qubits):
            if feature_idx < max_features:
                qc.rz(data_to_encode[feature_idx] * np.pi, i)
                feature_idx = (feature_idx + 1) % max_features
    
    for i in range(0, num_qubits-1, 2):
        qc.swap(i, i+1)
    
    for i in range(num_qubits):
        angle = np.pi / (2**(i % 5))
        qc.rz(angle, i)
    
    for i in range(num_qubits):
        for j in range(i+1, num_qubits):
            if (i+j) % 4 == 0:
                qc.cx(i, j)
    
    return qc

In [8]:
def quantum_encode_image(img_array, num_qubits=10):
    downsampled_r = resize_and_flatten(img_array[:,:,0], (8, 8))
    downsampled_g = resize_and_flatten(img_array[:,:,1], (8, 8))
    downsampled_b = resize_and_flatten(img_array[:,:,2], (8, 8))
    
    combined = np.concatenate([downsampled_r, downsampled_g, downsampled_b])
    normalized = (combined - np.min(combined)) / (np.max(combined) - np.min(combined) + 1e-10)
    
    return shor_inspired_quantum_circuit(normalized, num_qubits)

In [9]:
def resize_and_flatten(channel, size):
    from PIL import Image
    img = Image.fromarray((channel * 255).astype(np.uint8))
    img = img.resize(size)
    return np.array(img).flatten() / 255.0

In [10]:
def extract_quantum_features(qc, shots=8192):
    simulator = Aer.get_backend('qasm_simulator')
    qc_measure = qc.copy()
    qc_measure.measure_all()
    
    result = simulator.run(qc_measure, shots=shots).result()
    counts = result.get_counts()
    
    num_states = 2**qc.num_qubits
    probabilities = np.zeros(num_states)
    
    for bitstring, count in counts.items():
        try:
            idx = int(bitstring, 2)
            probabilities[idx] = count / shots
        except:
            continue
    
    top_k = 50
    indices = np.argsort(probabilities)[-top_k:]
    features = probabilities[indices]
    
    return features

In [11]:
def get_all_image_paths(dataset_path):
    image_paths = []
    labels = []
    
    for idx, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                img_path = os.path.join(class_path, img_file)
                image_paths.append(img_path)
                labels.append(idx)
    
    return np.array(image_paths), np.array(labels)

In [12]:
def split_dataset(image_paths, labels, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    np.random.seed(42)
    indices = np.random.permutation(len(image_paths))
    
    train_size = int(len(indices) * train_ratio)
    val_size = int(len(indices) * val_ratio)
    
    train_idx = indices[:train_size]
    val_idx = indices[train_size:train_size+val_size]
    test_idx = indices[train_size+val_size:]
    
    train_paths = image_paths[train_idx]
    train_labels = labels[train_idx]
    
    val_paths = image_paths[val_idx]
    val_labels = labels[val_idx]
    
    test_paths = image_paths[test_idx]
    test_labels = labels[test_idx]
    
    return (train_paths, train_labels), (val_paths, val_labels), (test_paths, test_labels)

In [13]:
def extract_features(image_path):
    img_array = preprocess_image(image_path)
    classical_features = extract_classical_features(img_array)
    qc = quantum_encode_image(img_array)
    quantum_features = extract_quantum_features(qc)
    return np.concatenate([classical_features, quantum_features])

In [14]:
def process_dataset(paths, labels, class_to_idx, batch_size=10):
    X = []
    y = []
    
    for i in range(0, len(paths), batch_size):
        batch_paths = paths[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]
        
        for img_path, label in zip(batch_paths, batch_labels):
            try:
                features = extract_features(img_path)
                X.append(features)
                y.append(label)
                print(f"Processed {img_path}")
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
    
    return np.array(X), np.array(y)

In [15]:
image_paths, labels = get_all_image_paths(dataset_path)
(train_paths, train_labels), (val_paths, val_labels), (test_paths, test_labels) = split_dataset(image_paths, labels)

print(f"Train: {len(train_paths)}, Validation: {len(val_paths)}, Test: {len(test_paths)}")

class_to_idx = {class_name: idx for idx, class_name in enumerate(classes)}
idx_to_class = {v: k for k, v in class_to_idx.items()}

Train: 1470, Validation: 315, Test: 315


In [16]:
image_paths, labels = get_all_image_paths(dataset_path)

In [1]:
X_train, y_train = process_dataset(train_paths, train_labels, class_to_idx)
X_val, y_val = process_dataset(val_paths, val_labels, class_to_idx)
X_test, y_test = process_dataset(test_paths, test_labels, class_to_idx)

NameError: name 'process_dataset' is not defined

In [2]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

NameError: name 'X_train' is not defined

In [47]:
! pip install tensorflow

  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.17.2-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
  Using cached grpcio-1.71.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached h5py-3.13.0-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached optree-0.14.1-cp312-cp312-win_amd64.whl.metadata (50 kB)
  Using cached Markdown-3.7-py3-

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(len(classes), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=3)
]

history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=25,
    batch_size=32,
    callbacks=callbacks
)

/Users/hrudaydivakaran/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


NameError: name 'X_train_scaled' is not defined

In [1]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.19.0-cp39-cp39-macosx_12_0_arm64.whl.metadata (4.0 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-1-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.4-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached termcolor-3.0.1-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.17.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (6.4 kB)
  Using cached grpcio-1.71.0-cp39-cp39-mac